In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Função para extrair dados de uma tabela específica
def extrair_dados(url):
    response = requests.get(url)
    if response.status_code != 200:
        return None  # Retorna None se a página não estiver disponível
    
    # Parseia o HTML da página usando Beautifulsoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Busca a tabela específica pela classe
    tabela = soup.find("table", {"class": "tb_base tb_dados"})
    
    if not tabela:
        return None  # Retorna None se a tabela não for encontrada
    
    # Extrai as linhas da tabela
    linhas = tabela.find_all("tr")

    # Inicializa lista para armazenar dados
    cabecalhos = [th.get_text(strip=True) for th in linhas[0].find_all("th")]
    dados = []
    
    # Itera sobre as linhas restantes para extrair os dados
    for linha in linhas[1:]:
        celulas = linha.find_all("td")
        if celulas: # Ignora linhas sem células
            valores = [td.get_text(strip=True) for td in celulas]
            dados.append(valores)
    
    return dados

In [51]:
# Lista de anos para iterar
anos = range(1970, 2024)  # De 1970 até 2023

# Dicionário para armazenar os dados por ano
dados_por_ano = {}

In [52]:
# Itera sobre os anos e extrai os dados
for ano in anos:
    url = f"http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao=opt_02"
    dados = extrair_dados(url)
    
    if dados:
        # Armazena os dados no dicionário, usando o ano como chave
        dados_por_ano[str(ano)] = [linha[1] for linha in dados]  # Apenas a coluna de quantidade
    else:
        print(f"Dados para o ano {ano} não disponíveis.")

In [53]:
# Cria o DataFrame final
if dados_por_ano:
    # Extrai os produtos (coluna fixa)
    produtos = [linha[0] for linha in dados]  # Assume que os produtos são os mesmos para todos os anos
    
    # Cria o DataFrame com os produtos como índice
    df = pd.DataFrame(dados_por_ano, index=produtos)
    df.index.name = "Produto"  # Define o nome da coluna de índice
    
    # Exibe o DataFrame
    print(df)
else:
    print("Nenhum dado disponível para os anos especificados.")

                                                 1970         1971  \
Produto                                                              
VINHO DE MESA                             217.208.604  154.264.651   
Tinto                                     174.224.052  121.133.369   
Branco                                        748.400    1.160.500   
Rosado                                     42.236.152   31.970.782   
VINHO FINO DE MESA (VINIFERA)              23.899.346   23.586.062   
Tinto                                       7.591.557    7.265.666   
Branco                                     15.562.889   15.655.709   
Rosado                                        744.900      664.687   
SUCO                                        1.097.771    2.296.654   
Suco de uva integral                        1.097.771    2.296.654   
Suco de uva concentrado                             -            -   
Suco de uva adoçado                                 -            -   
Suco de uva orgânico

In [54]:
# Salva o DataFrame em um arquivo CSV, se desejado
df.to_csv("producao_vinhos.csv", sep=";")